# Obtain NBA data

In [446]:
#%pip install nba_api
#%pip install beautifulsoup4
#%pip install lxml


In [447]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import playerawards
from nba_api.stats.endpoints import boxscoreadvancedv2
from nba_api.stats.endpoints import leaguegamefinder
from itertools import product
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import numpy as np
import time
import requests
import json

In [448]:
#pandas options
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Related to the data we will obtain then primarily from https://www.nba.com/stats through the NBA API package developed by Swar Patel https://github.com/swar/nba_api

Although we saw that for some correct game id's the nba api package give some errors, and in some cases we have empty cells of data. Then to overcome that and increase the quality of our data we will use also data from https://www.basketball-reference.com/

To do so first we need to map for each player their Basketball-Reference.com NBA player ID with their NBA.com player ID. This step will be done using sparql wikidata query service as you can see below.

### 0.1 Functions to obtain for each player a mapping between Basketball-Reference.com NBA player ID and NBA.com player ID

In [449]:
def missing_data(input_data):
    '''
    This function returns dataframe with information about the percentage of nulls in each column and the column data type.
    
    input: pandas df
    output: pandas df
    
    '''
    
    total = input_data.isnull().sum()
    percent = (input_data.isnull().sum()/input_data.isnull().count()*100)
    table = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
    types = []
    for col in input_data.columns: 
        dtype = str(input_data[col].dtype)
        types.append(dtype)
    table["Types"] = types
    return(pd.DataFrame(table))

In [450]:
def obtain_wiki_data(query):

    '''
    This function sends a query to the Wikidata SPARQL API and returns the response data as a dictionary.

    Parameters:
    query (str): A SPARQL query string to fetch data from Wikidata.

    Returns:
    dict: A dictionary containing the JSON response data from the API.
    '''


    # Define the Wikidata SPARQL API URL
    url = 'https://query.wikidata.org/sparql'

    # Send a GET request to the Wikidata SPARQL API with the provided query
    r = requests.get(url, params={'format': 'json', 'query': query})

    # Parse the JSON response and return it as a Python dictionary
    response_data = r.json()

    # Return the response data
    return response_data


In [451]:
def create_player_mapping_df(data_json):
    '''
        This function converts JSON data containing player id's (Wikidata id, NBA.com ID and Basketball Reference ID) into a pandas DataFrame.

        Parameters:
        data_json (dict): A dictionary containing JSON data with player information.

        Returns:
        pandas.DataFrame: A DataFrame with player information extracted from the JSON data.
    '''
    players = data_json['results']['bindings']

    players_list_to_df = []
    
    new_dict = {}
    
    for player in players:
        for key in player.keys():
            temp_dict = player[key].copy()
            new_dict[key] = temp_dict['value']   
        
        players_list_to_df.append(new_dict)
        new_dict = {}
    
    mapping_ids_df = pd.DataFrame(players_list_to_df)

    return mapping_ids_df
    

In [452]:
def obtain_player_mapping_df():
    '''
    This function retrieves player mapping id's (Wikidata id, NBA.com ID and Basketball Reference ID)  information from Wikidata 
    using a SPARQL query and returns it as a DataFrame.

    Returns:
    pandas.DataFrame: A DataFrame containing player mapping information, including Wikidata item IDs, Basketball Reference IDs, NBA IDs, 
    and player names.
    '''

    query = '''
    SELECT
      ?item ?itemLabel
      ?basketballRefID
      ?NBAID
    WHERE 
    {
      ?item wdt:P2685 ?basketballRefID.
      OPTIONAL{?item wdt:P3647 ?NBAID.}
    
    
            
      # change P1800 to another property        
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    '''
    data_json = obtain_wiki_data(query)
    mapping_df = create_player_mapping_df(data_json)

    return mapping_df
    
    

In [453]:
mapping_ids_df = obtain_player_mapping_df()

In [454]:
mapping_ids_df.to_csv('medium/mapping_ids_table.csv', index= False)

mapping_ids_df.head(10)

,item,basketballRefID,NBAID,itemLabel
0,http://www.wikidata.org/entity/Q24002,p/pankoan01,1950,Andy Panko
1,http://www.wikidata.org/entity/Q29342,t/thompkl01,202691,Klay Thompson
2,http://www.wikidata.org/entity/Q41421,j/jordami01,893,Michael Jordan
3,http://www.wikidata.org/entity/Q42079,w/winslri01,1984,Rickie Winslow
4,http://www.wikidata.org/entity/Q48842,e/ezelife01,203105,Festus Ezeli
5,http://www.wikidata.org/entity/Q60118,g/gallida01,201568,Danilo Gallinari
6,http://www.wikidata.org/entity/Q65039,w/waitedi01,203079,Dion Waiters
7,http://www.wikidata.org/entity/Q65594,n/ndongbo01,NaN,Boniface N'Dong
8,http://www.wikidata.org/entity/Q72353,i/ilgauzy01,980,Žydrūnas Ilgauskas
9,http://www.wikidata.org/entity/Q74005,f/favervi01,NaN,Vitor Faverani


## Part 1: Obtain Player data

### 1.1 Get player id's

With the code below we get the player id of each player that played in the NBA

In [455]:
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
nba_players = pd.DataFrame(nba_players)

In [456]:

nba_players.to_csv('medium/nba_players.csv', index= False)
nba_players.head(2)

,id,full_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False


### 1.2 Get player stats for each player 

The code below iterates through a list of NBA player IDs extracted from a DataFrame, fetching their career statistics and player-specific information using external API calls. For each player, it combines these dataframes by merging them based on a shared 'PLAYER_ID' column, adding player-specific details like name, draft year, and draft number to their career stats. The code collects these merged dataframes into a list, and after processing all player IDs, it concatenates them into a single comprehensive DataFrame called combined_career_data. This final DataFrame contains aggregated career statistics for all NBA players, enriched with individual player information, making it suitable for further analysis.

The running time of the commented part is around 3 h this is why we have stored the data obtained in a csv that you can find here []. So we don't have to run this part each time.

In [457]:
# Record the start time
start_time1 = time.time()

In [458]:
####   # CODE CELL
####   player_ids = nba_players['id']
####   
####   # Initialize an empty list to store player career dataframes
####   career_dataframes = []
####   
####   # Iterate through player IDs and fetch career stats
####   c = 1
####   for player_id in player_ids:
####   
####       if c % 100 == 0: print(f"Number of id's extracted: {c}")
####   
####       career = playercareerstats.PlayerCareerStats(player_id=player_id)
####       player_career = career.get_data_frames()[0]
####       player_info = commonplayerinfo.CommonPlayerInfo(player_id= player_id).get_data_frames()[0]
####   
####   
####       player_info = player_info.rename(columns={'PERSON_ID': 'PLAYER_ID'})
####   
####       player_info = player_info[['PLAYER_ID', 'DISPLAY_FIRST_LAST', 'DRAFT_YEAR', 'DRAFT_NUMBER']]
####   
####       player_career = pd.merge(player_career, player_info, on='PLAYER_ID', how='left')
####   
####   
####       career_dataframes.append(player_career)
####   
####   
####   
####       c += 1
####   
####       #if c >= 1: break
####   
####   
####   # Concatenate all player career dataframes into a single dataframe
####   combined_career_data = pd.concat(career_dataframes, ignore_index=True)
####   
####   # Now, combined_career_data contains career statistics for all players
####   

In [459]:
###   # CODE CELL
###   combined_career_data = combined_career_data[combined_career_data['TEAM_ID'] != 0]
###   
###   # Now we need to reorder columns change names and compute efg
###   
###   combined_career_data.to_csv('player_data_not_clean.csv')


In [460]:
combined_career_data = pd.read_csv('player_data_not_clean.csv')


In [461]:
# Record the end time
end_time1 = time.time()

# Calculate the elapsed time
elapsed_time1 = end_time1 - start_time1

# Print the elapsed time
print(f"Elapsed time to obtain player stats: {elapsed_time1} seconds")

Elapsed time to obtain player stats: 0.1706559658050537 seconds


### 1.3 Clean player stats data and deal with null values

First we modify SEASON_ID column to have in that column the start year of the season only for example for the 1990-1991 season now on will by indicated only by 1990.

We also get data only from 1983-1984 season until now since we don't have data for the all-star game before the 1983-1984 season.

In [462]:
player_stats_not_clean = combined_career_data.copy()

# Correct the Jerry Smith draft year error in NBA.com
player_stats_not_clean.loc[player_stats_not_clean['PLAYER_ID'] == 202536, 'DRAFT_YEAR'] = 2010

player_stats_not_clean['SEASON_ID'] = player_stats_not_clean['SEASON_ID'].str.split('-').str[0].astype(int)
player_stats_not_clean = player_stats_not_clean[player_stats_not_clean['SEASON_ID'] >= 1983]

player_stats_not_clean.head(5)


,Unnamed: 0,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,DISPLAY_FIRST_LAST,DRAFT_YEAR,DRAFT_NUMBER
0,0,76001,1990,0,1610612757,POR,23.00,43,0.00,290.00,55,116,0.47,0.00,0.00,0.00,25,44,0.57,27.00,62.00,89.00,12,4.00,12.00,22.00,39,135,Alaa Abdelnaby,1990,25
1,1,76001,1991,0,1610612757,POR,24.00,71,1.00,934.00,178,361,0.49,0.00,0.00,0.00,76,101,0.75,81.00,179.00,260.00,30,25.00,16.00,66.00,132,432,Alaa Abdelnaby,1990,25
2,2,76001,1992,0,1610612749,MIL,25.00,12,0.00,159.00,26,56,0.46,0.00,1.00,0.00,12,16,0.75,12.00,25.00,37.00,10,6.00,4.00,13.00,24,64,Alaa Abdelnaby,1990,25
3,3,76001,1992,0,1610612738,BOS,25.00,63,52.00,1152.00,219,417,0.53,0.00,0.00,0.00,76,100,0.76,114.00,186.00,300.00,17,19.00,22.00,84.00,165,514,Alaa Abdelnaby,1990,25
4,5,76001,1993,0,1610612738,BOS,26.00,13,0.00,159.00,24,55,0.44,0.00,0.00,0.00,16,25,0.64,12.00,34.00,46.00,3,2.00,3.00,17.00,20,64,Alaa Abdelnaby,1990,25


Now we use the missing data function to know which columns have missig data. And we see that FG_PCT, FG3_PCT and FT_PCT can be computed using the other columns. And DRAFT_NUMBER makes sense since some players may be undrafted

In [463]:
missing = missing_data(player_stats_not_clean)

missing = missing[missing['Total'] != 0]

missing

,Total,Percent,Types
FG_PCT,1,0.01,float64
FG3_PCT,53,0.27,float64
FT_PCT,10,0.05,float64
DRAFT_NUMBER,118,0.60,object


#### 1.3.1 Deal with rows without draft year

With respect to the column 'DRAFT_YEAR,' we know that some players can be undrafted. However, we also know that the last opportunity to be drafted is the calendar year when the player reaches the age of 22. Therefore, for those undrafted players, we will set their 'DRAFT_YEAR' as the year when the player reaches the age of 22 if the player started to play in the NBA when he has 23 years old or after. If the player is undrafted but started to play in the NBA before reaching the 22 years old we set their 'DRAFT_YEAR' as the year of their first season in the NBA

FUNCTIONS

In [464]:
def obtain_players_information(players_list):
    '''
    This function obtains detailed player information for a list of player IDs and returns it as a concatenated pandas DataFrame.

    Parameters:
    players_list (list): A list of player IDs.

    Returns:
    pandas.DataFrame: A DataFrame containing detailed player information for all players in the input list.
    '''

    players_info_dfs = []
    c = 1
    for player in players_list:

        if c % 50 == 0: print(f"Number of id's extracted: {c}")
        player_info = commonplayerinfo.CommonPlayerInfo(player_id = player).get_data_frames()[0]

        players_info_dfs.append(player_info)

        c += 1

    return pd.concat(players_info_dfs, ignore_index=True)

    

In [465]:
def transform_player_info_df_to_estimate_draft_year(players_info_df):

    '''
    This function transforms a player information DataFrame to estimate the draft year for each player. Useful for undrafted players 
    to know if they were drafted in which year they would have been drafted.

    Parameters:
    players_info_df (pandas.DataFrame): A DataFrame containing player information, including player IDs (PERSON_ID) and birthdates (BIRTHDATE).

    Returns:
    pandas.DataFrame: A transformed DataFrame with estimated draft years for each player.
    '''


    df = players_info_df.copy()

    df = df[['PERSON_ID', 'BIRTHDATE']]
    df['BIRTHDATE'] = pd.to_datetime(df['BIRTHDATE'])

    df['BIRTH_YEAR'] = df['BIRTHDATE'].dt.year
    df['YEAR_TURNING_22'] = df['BIRTH_YEAR'] + 22
    df = df.rename(columns={'PERSON_ID': 'PLAYER_ID'})

    return df

In [466]:
def estimate_draft_year_for_undrafted_players(undrafted_players_df1, undrafted_players_info_df1):

    '''
    This function estimates the draft year for undrafted NBA players based on their first NBA season and their 22 years old calendar year.

    Parameters:
    undrafted_players_df1 (pandas.DataFrame): A DataFrame containing information about undrafted players, including player IDs (PLAYER_ID) and NBA season IDs (SEASON_ID).
    undrafted_players_info_df1 (pandas.DataFrame): A DataFrame containing player information, including player IDs (PLAYER_ID), birth years, and estimated draft years.

    Returns:
    pandas.DataFrame: A DataFrame with estimated draft years for undrafted players.
    '''

    undrafted_players_df = undrafted_players_df1.copy()

    undrafted_players_info_df = undrafted_players_info_df1.copy()


    
    undrafted_players_df['SEASON_ID'] = undrafted_players_df['SEASON_ID'].astype(int)

    first_NBA_season = undrafted_players_df.groupby('PLAYER_ID')['SEASON_ID'].min().reset_index()
    first_NBA_season = first_NBA_season.rename(columns={'SEASON_ID': 'FIRST_NBA_SEASON'})

    draft_year_estimation_df = pd.merge(undrafted_players_info_df, first_NBA_season, on='PLAYER_ID', how='inner')

    draft_year_estimation_df['DRAFT_YEAR'] = draft_year_estimation_df.apply(lambda row: min(row['YEAR_TURNING_22'], row['FIRST_NBA_SEASON']), axis=1)


    return draft_year_estimation_df

    

In [467]:
def apply_replacements(row, estimated_draft_year_dict):

    '''
    This function applies replacements to estimated draft years for specific players if available in a dictionary 
    (Usually the dictionary contains a list of Undrafted players).

    Parameters:
    row (pandas.Series): A row from a DataFrame containing player information.
    estimated_draft_year_dict (dict): A dictionary containing player IDs as keys and replacement draft years as values.

    Returns:
    int: The estimated draft year for the player, possibly replaced with a specified value from the dictionary.
    '''
    

    if (row['DRAFT_YEAR'] is None) and (row['PLAYER_ID'] in undf_draft_year_mapping):

        return estimated_draft_year_dict[row['PLAYER_ID']]

    else:
        return row['DRAFT_YEAR']

ACTIONS

Get players that don't have draft year. Obtain their player information including their birthdate.

In [468]:
undf = player_stats_not_clean[player_stats_not_clean['DRAFT_YEAR'] == 'Undrafted']

undrafted_players = set(list(undf['PLAYER_ID']))

In [469]:
undf_info = obtain_players_information(undrafted_players)

Number of id's extracted: 50
Number of id's extracted: 100
Number of id's extracted: 150
Number of id's extracted: 200
Number of id's extracted: 250
Number of id's extracted: 300
Number of id's extracted: 350
Number of id's extracted: 400
Number of id's extracted: 450
Number of id's extracted: 500
Number of id's extracted: 550
Number of id's extracted: 600
Number of id's extracted: 650
Number of id's extracted: 700
Number of id's extracted: 750
Number of id's extracted: 800


Add a column to know in which year each of the undrafted players has 22 years old. And the estimated draft year will be the min(year turning 22, year of their first nba season)

In [470]:
undf_info2 = transform_player_info_df_to_estimate_draft_year(undf_info).copy()

In [471]:
undf_info3 = estimate_draft_year_for_undrafted_players(undf, undf_info2)

In [472]:
player_stats_not_clean2 = player_stats_not_clean.copy()

In [473]:
player_stats_not_clean2['DRAFT_YEAR'] = player_stats_not_clean2['DRAFT_YEAR'].replace('Undrafted', None)
player_stats_not_clean2['DRAFT_NUMBER'] = player_stats_not_clean2['DRAFT_NUMBER'].replace('Undrafted', None)

In [474]:
missing = missing_data(player_stats_not_clean2)

missing = missing[missing['Total'] != 0]

missing

,Total,Percent,Types
FG_PCT,1,0.01,float64
FG3_PCT,53,0.27,float64
FT_PCT,10,0.05,float64
DRAFT_YEAR,3050,15.48,object
DRAFT_NUMBER,3168,16.08,object


With the estimated draft year for each undrafted player computed. We create a mapping to assign that estimated draft year in the corresponding row of the original dataset

In [475]:
undf_draft_year_mapping = undf_info3.set_index('PLAYER_ID')['DRAFT_YEAR'].to_dict()


In [476]:
player_stats_not_clean2['DRAFT_YEAR'] = player_stats_not_clean2.apply(lambda row: apply_replacements(row, undf_draft_year_mapping), axis= 1)

Now we only have null values on FG_PCT, FG_PCT and FT_PCT. And on DRAFT_NUMBER but those null values refer to undrafted players.

In [477]:
missing = missing_data(player_stats_not_clean2)

missing = missing[missing['Total'] != 0]

missing

,Total,Percent,Types
FG_PCT,1,0.01,float64
FG3_PCT,53,0.27,float64
FT_PCT,10,0.05,float64
DRAFT_NUMBER,3168,16.08,object


#### 1.3.2 Compute FG2M, FG2A, FG2_PCT, FG_PCT, FG3_PCT and FT_PCT and EFG

In [478]:
def safe_divide(numerator, denominator):
    '''
    Safely divides two numbers, handling the case where the denominator is zero.

    Parameters:
    numerator (float or int): The number to be divided.
    denominator (float or int): The number to divide by.

    Returns:
    float: The result of the division if the denominator is not zero, otherwise 0.
    '''
    if denominator == 0:
        return 0
    else:
        return numerator / denominator

Now to solve null values in percentage rows. It's straightforward since we have the values of shots made and attempted by zone. And finally we also have computed statistics for FG2 zone and EFG the Effective Field Goal Percentage.

In [479]:
player_stats_not_clean3 = player_stats_not_clean2.copy()

In [480]:
player_stats_not_clean3["FG2M"] = player_stats_not_clean3["FGM"] - player_stats_not_clean3["FG3M"]
player_stats_not_clean3["FG2A"] = player_stats_not_clean3["FGA"] - player_stats_not_clean3["FG3A"]

zone_metrics = ['FG', 'FG2', 'FG3', 'FT']

for zone in zone_metrics:

    player_stats_not_clean3[f"{zone}_PCT"] = player_stats_not_clean3.apply(lambda row: safe_divide(row[f"{zone}M"], row[f"{zone}A"]), axis=1)


player_stats_not_clean3["EFG"] = (player_stats_not_clean3["FGM"] + 0.5 * player_stats_not_clean3["FG3M"]) / player_stats_not_clean3["FGA"]

player_stats_not_clean3['EFG'].fillna(0, inplace=True)

In [481]:
missing = missing_data(player_stats_not_clean3)

missing = missing[missing['Total'] != 0]

missing

,Total,Percent,Types
DRAFT_NUMBER,3168,16.08,object


Now we only need to format properly the player stats dataframe

In [482]:
column_rename_mapping = {'PLAYER_ID': 'nbapersonid', 
                     'SEASON_ID': 'season',  
                     'TEAM_ID': 'nbateamid',
                     'TEAM_ABBREVIATION': 'team', 
                     'PLAYER_AGE': 'playerage', 
                     'GP': 'games', 
                     'GS': 'games_start', 
                     'MIN': 'mins', 
                     'FGM': 'fgm', 
                     'FGA': 'fga',
                     'FG_PCT': 'fgp', 
                     'FG3M': 'fgm3', 
                     'FG3A': 'fga3', 
                     'FG3_PCT': 'fgp3', 
                     'FTM': 'ftm', 
                     'FTA': 'fta', 
                     'FT_PCT': 'ftp', 
                     'OREB': 'off_reb',
                     'DREB': 'def_reb', 
                     'REB': 'tot_reb', 
                     'AST': 'ast', 
                     'STL': 'steals', 
                     'BLK': 'blocks', 
                     'TOV': 'tov', 
                     'PF': 'tot_fouls', 
                     'PTS': 'points',
                     'DISPLAY_FIRST_LAST': 'player', 
                     'DRAFT_YEAR': 'draftyear', 
                     'DRAFT_NUMBER': 'draftpick', 
                     'FG2M': 'fgm2', 
                     'FG2A': 'fga2',
                     'FG2_PCT': 'fgp2', 
                     'EFG': 'efg'}

player_stats_not_clean3.rename(columns=column_rename_mapping, inplace=True)

In [483]:
column_names_array = [
    "nbapersonid", "player", "draftyear", "draftpick", "season", "nbateamid", "team",
    "games", "games_start", "mins", "fgm", "fga", "fgp", "fgm3", "fga3", "fgp3", "fgm2", "fga2",
    "fgp2", "efg", "ftm", "fta", "ftp", "off_reb", "def_reb", "tot_reb", "ast", "steals", "blocks",
    "tov", "tot_fouls", "points"
]


player_stats_not_clean3 = player_stats_not_clean3[column_names_array]

In [484]:
player_stats = player_stats_not_clean3.copy()

In [485]:
player_stats.head(100).to_csv('medium/player_stats2.csv', index= False)
player_stats.head(5)

,nbapersonid,player,draftyear,draftpick,season,nbateamid,team,games,games_start,mins,fgm,fga,fgp,fgm3,fga3,fgp3,fgm2,fga2,fgp2,efg,ftm,fta,ftp,off_reb,def_reb,tot_reb,ast,steals,blocks,tov,tot_fouls,points
0,76001,Alaa Abdelnaby,1990,25,1990,1610612757,POR,43,0.00,290.00,55,116,0.47,0.00,0.00,0.00,55.00,116.00,0.47,0.47,25,44,0.57,27.00,62.00,89.00,12,4.00,12.00,22.00,39,135
1,76001,Alaa Abdelnaby,1990,25,1991,1610612757,POR,71,1.00,934.00,178,361,0.49,0.00,0.00,0.00,178.00,361.00,0.49,0.49,76,101,0.75,81.00,179.00,260.00,30,25.00,16.00,66.00,132,432
2,76001,Alaa Abdelnaby,1990,25,1992,1610612749,MIL,12,0.00,159.00,26,56,0.46,0.00,1.00,0.00,26.00,55.00,0.47,0.46,12,16,0.75,12.00,25.00,37.00,10,6.00,4.00,13.00,24,64
3,76001,Alaa Abdelnaby,1990,25,1992,1610612738,BOS,63,52.00,1152.00,219,417,0.53,0.00,0.00,0.00,219.00,417.00,0.53,0.53,76,100,0.76,114.00,186.00,300.00,17,19.00,22.00,84.00,165,514
4,76001,Alaa Abdelnaby,1990,25,1993,1610612738,BOS,13,0.00,159.00,24,55,0.44,0.00,0.00,0.00,24.00,55.00,0.44,0.44,16,25,0.64,12.00,34.00,46.00,3,2.00,3.00,17.00,20,64


## Part 2: Obtain Awards Data

### 2.1 Get Awards Data

The code below processes a list of NBA player IDs by iteratively fetching their individual award data using an external API. It starts with an empty DataFrame named combined_df. For each player, it retrieves their award information and appends it to the combined_df using the pd.concat function, which combines multiple DataFrames. The resulting combined_df accumulates all the award data for NBA players, allowing for a comprehensive analysis of awards received by players. 

In [486]:
# Record the start time
start_time2 = time.time()

The crawling process commented below takes too long (1 h aprox) this is why we've done only one time and store the awards data obtained in a csv.

In [487]:
### # CODE CELL
### # Create an empty DataFrame to store the combined awards data
### combined_df = pd.DataFrame()
### 
### # Iterate through player IDs and append their awards DataFrames
### c = 1
### for player_id in player_ids:
###     if c % 100 == 0: print(f"Number of id's extracted: {c}")
###     player_awards = playerawards.PlayerAwards(player_id=player_id)
###     awards_df = player_awards.get_data_frames()[0]
###     combined_df = pd.concat([combined_df, awards_df], ignore_index=True)
### 
###     c += 1
### 
###     #if c == 15: break

In [488]:
### # CODE CELL
### combined_df.to_csv('awards_not_clean.csv')

In [489]:
combined_df = pd.read_csv('awards_not_clean.csv')

In [490]:
# Record the end time
end_time2 = time.time()

# Calculate the elapsed time
elapsed_time2 = end_time2 - start_time2

# Print the elapsed time
print(f"Elapsed time to obtain player awards: {elapsed_time2} seconds")

Elapsed time to obtain player awards: 0.08203411102294922 seconds


### 2.2 Get players that played an all_star_game

#### Methods to collect all star rosters

In [491]:
def get_all_star_team_ids(all_star_game_id):
    '''
    Retrieves the team IDs participating in an NBA All-Star game based on the game ID.

    Parameters:
    all_star_game_id (str): The unique identifier for a concrete NBA All-Star game.

    Returns:
    list: A list of team IDs (int) participating in the specified All-Star game.
    '''

    all_star_game_year = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id = all_star_game_id).get_data_frames()[0]

    all_star_teams = all_star_game_year[['TEAM_ID', 'TEAM_ABBREVIATION']].drop_duplicates()

    return list(all_star_teams['TEAM_ID'])
    

In [492]:
def get_all_star_games(all_star_team_id):
    '''
    Retrieves NBA All-Star games in which a specific team has participated based on the team's ID.

    Parameters:
    all_star_team_id (int): The unique identifier for the NBA team.

    Returns:
    pandas.DataFrame: A DataFrame containing information about the All-Star games in which the specified team has played.
    '''

    # Query for games where an all star team were playing
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable= all_star_team_id)
    # The first DataFrame of those returned is what we want.
    all_star_games = gamefinder.get_data_frames()[0]

    return all_star_games
    

In [493]:
def format_all_star_games_df(all_star_games_df):

    '''
    Formats and preprocesses an NBA All-Star games DataFrame.

    Parameters:
    all_star_games_df (pandas.DataFrame): The DataFrame containing information about NBA All-Star games.

    Returns:
    pandas.DataFrame: The formatted DataFrame with specific columns.
    '''

    all_star_games_df['GAME_DATE'] = pd.to_datetime(all_star_games_df['GAME_DATE'])
    
    all_star_games_df['YEAR'] = all_star_games_df['GAME_DATE'].dt.year
    all_star_games_df['MONTH'] = all_star_games_df['GAME_DATE'].dt.month
    all_star_games_df['DAY'] = all_star_games_df['GAME_DATE'].dt.day



    all_star_games_df['SEASON'] = all_star_games_df.apply(lambda row: f"{row['YEAR']-1}-{row['YEAR']}" if row['MONTH'] <= 7 else f"{row['YEAR']}-{row['YEAR']+1}", axis=1)

    all_star_games_df = all_star_games_df[['SEASON_ID', 'MATCHUP', 'GAME_ID', 'GAME_DATE', 'SEASON']]

    return all_star_games_df
    

In [494]:
def obtain_all_star_games_ids(all_star_games_df):
    '''
    Obtains a dictionary mapping NBA All-Star game seasons to their corresponding game IDs.

    Parameters:
    all_star_games_df (pandas.DataFrame): The DataFrame containing information about NBA All-Star games.

    Returns:
    dict: A dictionary where keys are season names and values are the corresponding game IDs.
    '''
    all_star_game_ids = {}
    for index, row in all_star_games_df.iterrows():
        if row['GAME_ID'].startswith('00'):
            
            all_star_game_ids[row['SEASON']] = row['GAME_ID']

    return all_star_game_ids

In [495]:
def get_all_star_roster_from_NBA_com(gameID, season):

    '''
    Retrieves the roster of NBA All-Star players for a specific game from NBA.com.

    Parameters:
    gameID (str): The unique identifier for the NBA All-Star game.
    season (str): The season for which the All-Star game roster is being retrieved.

    Returns:
    pandas.DataFrame: A DataFrame containing information about All-Star players in the specified game.
    '''

    roster = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id = gameID).get_data_frames()[0]


    roster = roster[['GAME_ID', 'PLAYER_ID', 'PLAYER_NAME']]

    roster['SEASON'] = season

    roster['IS_ALLSTAR'] = True

    return roster

In [496]:
def format_all_star_basketballRef_df(basketballRef_df, season, game_id):

    '''
    Formats and preprocesses a DataFrame obtained from Basketball-Reference with NBA All-Star player information. The goal is having a df
    with the same format as it would have been obtained from NBA.com

    Parameters:
    basketballRef_df (pandas.DataFrame): The DataFrame containing NBA All-Star player information from Basketball-Reference.
    season (str): The season for which the All-Star roster is being formatted.
    game_id (str): The unique identifier for the NBA All-Star game.

    Returns:
    pandas.DataFrame: A formatted DataFrame containing information about All-Star players.
    '''

    roster_df = pd.DataFrame(columns=['GAME_ID', 'PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'IS_ALLSTAR'])

    roster_df['PLAYER_ID'] = basketballRef_df['NBAID']
    roster_df['PLAYER_NAME'] = basketballRef_df['itemLabel']
    roster_df['PLAYER_NAME'] = basketballRef_df['itemLabel']
    roster_df['GAME_ID'] = game_id
    roster_df['SEASON'] = season
    roster_df['IS_ALLSTAR'] = True

    return roster_df

In [497]:
def get_all_star_roster_from_basketball_reference(season, mapping_ids_df):
    '''
    Retrieves the NBA All-Star roster from Basketball-Reference for a specific season.

    Parameters:
    season (str): The season for which the All-Star roster is being retrieved.
    mapping_ids_df (pandas.DataFrame): A DataFrame containing a df that maps each NBA.com player id with the corresponding Basketball Reference
    player id.

    Returns:
    pandas.DataFrame: A DataFrame containing player information for All-Star players in the specified season.
    '''

    try:
        season = season.split('-')[1]
    except:
        print(f'ERROR NOT VALID SEASON FORMAT FOR SEASON: {season}')
        return None
    
    # Construct the URL based on the season
    url = f'https://www.basketball-reference.com/allstar/NBA_{season}.html'

    # Send an HTTP GET request to the URL
    response = requests.get(url)
    html_content = response.text

    # Parse the HTML content with lxml
    tree = html.fromstring(html_content)

    # Define your XPath expression to extract player IDs
    xpath_expression = "//div[contains(@class, 'table_wrapper')]//th[@data-stat='player']/a/@href"

    # Use XPath to select player IDs
    selected_elements = tree.xpath(xpath_expression)

    # Clean up the selected_elements (remove '/players/' and '.html')
    selected_elements = [player.replace("/players/", "").replace(".html", "") for player in selected_elements]

    # Filter the mapping DataFrame based on the selected_elements
    filtered_df = mapping_ids_df[mapping_ids_df['basketballRefID'].isin(selected_elements)]

    return filtered_df

In [498]:
def obtain_and_format_all_star_BRef_roster(season, mapping_ids_df, all_star_game_ids):
    '''
    Obtains and formats the NBA All-Star roster from Basketball-Reference for a specific season.

    Parameters:
    season (str): The season for which the All-Star roster is being retrieved and formatted.
    mapping_ids_df (pandas.DataFrame): A DataFrame containing player mapping information.
    all_star_game_ids (dict): A dictionary mapping NBA All-Star game seasons to their corresponding game IDs.

    Returns:
    pandas.DataFrame: A formatted DataFrame containing player information for All-Star players in the specified season.
    '''
    
    roster_bref = get_all_star_roster_from_basketball_reference(season, mapping_ids_df)
    roster_bref = format_all_star_basketballRef_df(roster_bref, season, all_star_game_ids[season])

    return roster_bref

In [499]:
def obtain_all_star_rosters(all_star_game_ids, mapping_ids_df):

    '''
    Obtains NBA All-Star rosters for multiple seasons, combining data from different sources.

    Parameters:
    all_star_game_ids (dict): A dictionary mapping NBA All-Star game seasons to their corresponding game IDs.
    mapping_ids_df (pandas.DataFrame): A DataFrame containing player mapping information.

    Returns:
    pandas.DataFrame: A combined DataFrame containing player information for All-Star players across multiple seasons.
    '''
    
    all_star_games_rosters = []

    for season in all_star_game_ids.keys():
        try:
            roster = get_all_star_roster_from_NBA_com(all_star_game_ids[season], season)
            all_star_games_rosters.append(roster)
            
                


        except:
            roster = obtain_and_format_all_star_BRef_roster(season, mapping_ids_df, all_star_game_ids)
            all_star_games_rosters.append(roster)


    all_star_games_rosters = pd.concat(all_star_games_rosters, ignore_index=True)

    return all_star_games_rosters
    

#### Actions to collect all star rosters

First we get the team id of one of the two teams that play the All Star game. 

* East: 1610616833
* West: 1610616834

To do that we just call the nba api with the game id of the last all star game to get the boxscore. And with that we can obtain the two id's


In [500]:
# https://www.nba.com/game/gns-vs-lbn-0032200001/box-score#box-score
all_star_game_id_2023 = '0032200001'
t_id = get_all_star_team_ids(all_star_game_id_2023)[0]

Now we need the game id of each all star game. To obtain them we just need to  call the leaguegamefinder nba-api endpoint with the team id of one of the two teams that play the all star game. In this case we called the leaguegamefinder nba-api endpoint with the team id of East. And as you can see below now we have all the all star games played since 1984 with their corresponding game id.

In [501]:
all_star_games = get_all_star_games(t_id)
all_star_games = format_all_star_games_df(all_star_games)
all_star_games.to_csv('medium/all-star game ids.csv', index= False)
all_star_games.head(8)

,SEASON_ID,MATCHUP,GAME_ID,GAME_DATE,SEASON
0,32022,GNS @ LBN,0032200001,2023-02-19,2022-2023
1,32021,DRT @ LBN,0032100001,2022-02-20,2021-2022
2,32020,DRT vs. LBN,0032000001,2021-03-07,2020-2021
3,32019,GNS vs. LBN,0031900001,2020-02-16,2019-2020
4,32019,GNS vs. LBN,2531900001,2020-01-23,2019-2020
5,32018,GNS vs. LBN,0031800001,2019-02-17,2018-2019
6,32017,LBN @ STP,0031700001,2018-02-18,2017-2018
7,32016,EST @ WST,0031600001,2017-02-19,2016-2017


With that we create a dictionary to store for each season the corresponding game id

In [502]:
all_star_game_ids = obtain_all_star_games_ids(all_star_games)
print(all_star_game_ids)

{'2022-2023': '0032200001', '2021-2022': '0032100001', '2020-2021': '0032000001', '2019-2020': '0031900001', '2018-2019': '0031800001', '2017-2018': '0031700001', '2016-2017': '0031600001', '2015-2016': '0031500001', '2014-2015': '0031400001', '2013-2014': '0031300001', '2012-2013': '0031200001', '2011-2012': '0031100001', '2010-2011': '0031000001', '2009-2010': '0030900001', '2008-2009': '0030800001', '2007-2008': '0030700001', '2006-2007': '0030600001', '2005-2006': '0030500001', '2004-2005': '0030400001', '2003-2004': '0030300001', '2002-2003': '0030200001', '2001-2002': '0030100001', '2000-2001': '0030000001', '1999-2000': '0039900001', '1997-1998': '0039700001', '1996-1997': '0039600001', '1995-1996': '0039500001', '1994-1995': '0039400001', '1993-1994': '0039310001', '1992-1993': '0039210001', '1991-1992': '0039100002', '1990-1991': '0039000002', '1989-1990': '0038900002', '1988-1989': '0038800002', '1987-1988': '0038700001', '1986-1987': '0038600001', '1985-1986': '0038500001', 

So now we can obtain the roster of each all star game since 1984. We've done in two ways for those seasons that nba-api gives us the data without any problem we call boxscoreadvancedv2 endpoint and obtain the roster.

And for those seasons and game id's that the nba-api package were boxscoreadvancedv2 does not work. We go to basketball reference for example https://www.basketball-reference.com/allstar/NBA_1998.html and get the Basketball-Reference.com NBA player ID of each player scrapping the web page with beautiful soup and lxml. Then we map the Basketball-Reference.com NBA player ID obtained with their corresponding NBA.com player id. Through the mapping obtained from wikidata.

In [503]:
all_star_games_rosters = obtain_all_star_rosters(all_star_game_ids, mapping_ids_df)



In [504]:
all_star_games_rosters.to_csv('medium/all_star_rosters.csv', index= False)

all_star_games_rosters.head(5)

,GAME_ID,PLAYER_ID,PLAYER_NAME,SEASON,IS_ALLSTAR
0,0032200001,1628369,Jayson Tatum,2022-2023,True
1,0032200001,203507,Giannis Antetokounmpo,2022-2023,True
2,0032200001,1628374,Lauri Markkanen,2022-2023,True
3,0032200001,1628378,Donovan Mitchell,2022-2023,True
4,0032200001,1629630,Ja Morant,2022-2023,True


### 2.3 Modify awards data to include all star appearances

In [505]:
def print_unique_values_counts(df, columns_to_check):
    '''
    Prints the unique values and their counts for specified columns in a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame to analyze.
    columns_to_check (list): A list of column names to check and print unique values and counts.

    Returns:
    None (prints the results to the console).
    '''
    for column in columns_to_check:
        unique_values_counts = df[column].value_counts()
        print(f"Column '{column}':")
        for value, count in unique_values_counts.items():
            print(f"Value '{value}': {count} times")
        print()

In [506]:
def clean_awards_data(df):
    '''
    Cleans and preprocesses columns in an awards DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing awards data.

    Returns:
    pandas.DataFrame: The cleaned DataFrame with modified columns.
    '''
    
    # Clean the 'SEASON' column
    df['SEASON'] = df['SEASON'].str.split('-').str[0].astype(int)

    # Clean the 'ALL_NBA_TEAM_NUMBER' column
    df['ALL_NBA_TEAM_NUMBER'] = df['ALL_NBA_TEAM_NUMBER'].replace('(null)', np.nan)
    df['ALL_NBA_TEAM_NUMBER'] = df['ALL_NBA_TEAM_NUMBER'].replace(np.nan, 0)
    df['ALL_NBA_TEAM_NUMBER'] = df['ALL_NBA_TEAM_NUMBER'].astype(int)

    return df

In [507]:
def extract_all_nba_teams(df):
    '''
    Extracts All-NBA team players from an awards DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing awards data.

    Returns:
    pandas.DataFrame: A DataFrame containing All-NBA team  players.
    '''

    # Filter rows where DESCRIPTION is 'All-NBA'
    allNBA = df[df['DESCRIPTION'] == 'All-NBA'].copy()

    # Create new columns for All-NBA team memberships
    allNBA['All_NBA_First_Team'] = np.where(allNBA['ALL_NBA_TEAM_NUMBER'] == 1, True, False)
    allNBA['All_NBA_Second_Team'] = np.where(allNBA['ALL_NBA_TEAM_NUMBER'] == 2, True, False)
    allNBA['All_NBA_Third_Team'] = np.where(allNBA['ALL_NBA_TEAM_NUMBER'] == 3, True, False)

    # Select relevant columns
    allNBA = allNBA[['PERSON_ID', 'SEASON', 'All_NBA_First_Team', 'All_NBA_Second_Team', 'All_NBA_Third_Team']]

    # Rename columns
    allNBA.rename(columns={'PERSON_ID': 'nbapersonid', 'SEASON': 'season'}, inplace=True)

    return allNBA

In [508]:
def extract_mvp_nba(df):
    '''
    Extracts NBA Most Valuable Players from an awards DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing awards data.

    Returns:
    pandas.DataFrame: A DataFrame containing all players that have been selected as MVP in an NBA season from the awards df.
    '''
    # Filter rows where DESCRIPTION is 'NBA Most Valuable Player'
    mvpNBA = df[df['DESCRIPTION'] == 'NBA Most Valuable Player'].copy()

    # Rename columns
    mvpNBA.rename(columns={'PERSON_ID': 'nbapersonid', 'SEASON': 'season'}, inplace=True)

    # Select relevant columns
    mvpNBA = mvpNBA[['nbapersonid', 'season']]

    # Add a 'Most_Valuable_Player' column with True values
    mvpNBA['Most_Valuable_Player'] = True

    return mvpNBA

In [509]:
def extract_dpoy_nba(df):
    '''
    Same than function before extract_mvp_nba(df) but in this case getting the players that have been selected as Defensive Player of the Year
    in an NBA season from an awards df.
    '''
    # Filter rows where DESCRIPTION is 'NBA Defensive Player of the Year'
    dpoyNBA = df[df['DESCRIPTION'] == 'NBA Defensive Player of the Year'].copy()

    # Rename columns
    dpoyNBA.rename(columns={'PERSON_ID': 'nbapersonid', 'SEASON': 'season'}, inplace=True)

    # Select relevant columns
    dpoyNBA = dpoyNBA[['nbapersonid', 'season']]

    # Add a 'Defensive_Player_of_The_Year' column with True values
    dpoyNBA['Defensive_Player_of_The_Year'] = True

    return dpoyNBA

In [510]:
def transform_all_star_rosters(df):
    '''
    Transforms and preprocesses an NBA All-Star rosters DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame containing NBA All-Star roster information.

    Returns:
    pandas.DataFrame: A transformed DataFrame with modified columns.
    '''
    
    # Create a copy of the DataFrame
    all_star_rosters2 = df.copy()

    # Clean the 'SEASON' column
    all_star_rosters2['SEASON'] = all_star_rosters2['SEASON'].str.split('-').str[0].astype(int)

    # Select relevant columns
    all_star_rosters2 = all_star_rosters2[['PLAYER_ID', 'SEASON', 'IS_ALLSTAR']]

    # Rename columns
    all_star_rosters2.rename(columns={'PLAYER_ID': 'nbapersonid', 'SEASON': 'season', 'IS_ALLSTAR': 'all_star_game'}, inplace=True)

    # Convert 'nbapersonid' column to integer
    all_star_rosters2['nbapersonid'] = all_star_rosters2['nbapersonid'].astype(int)

    return all_star_rosters2


In [511]:
def convert_columns_to_binary(df):
    '''
    Converts specified columns to binary format (0 or 1) in a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame to convert.

    Returns:
    pandas.DataFrame: A DataFrame with specified columns converted to binary format.
    '''
    result_df = df.copy()
    cols = list(result_df.columns)
    cols.remove('nbapersonid')
    cols.remove('season')
    for col in cols:
        result_df[col] = result_df[col].fillna(False)
        if col != 'all_star_game':
            result_df[col] = result_df[col].astype(int)
    return result_df

First we get the awards data as we have obtained from the playerawards endpoint using the nba-api python package

In [512]:
awards_not_clean = combined_df.copy()

awards_not_clean.head(3)

,Unnamed: 0,PERSON_ID,FIRST_NAME,LAST_NAME,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3
0,0,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,2,1969-70,NaN,NaN,1610612749,Award,Kia Motors,KIADT,NaN
1,1,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,2,1970-71,NaN,NaN,1610612749,Award,Kia Motors,KIADT,NaN
2,2,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,1,1973-74,NaN,NaN,1610612749,Award,Kia Motors,KIADT,NaN


After that we convert ALL_NBA_TEAM_NUMBER column to numerical and we set values different from 1, 2 or 3 to 0 indicating that the player in that season it has not been selected in any of the All NBA teams.

In [513]:
awards_not_clean = clean_awards_data(awards_not_clean)

unique_values = awards_not_clean['ALL_NBA_TEAM_NUMBER'].unique()

# Print the unique values
print(unique_values)


awards_not_clean.head(1)

[2 1 0 3]


,Unnamed: 0,PERSON_ID,FIRST_NAME,LAST_NAME,TEAM,DESCRIPTION,ALL_NBA_TEAM_NUMBER,SEASON,MONTH,WEEK,CONFERENCE,TYPE,SUBTYPE1,SUBTYPE2,SUBTYPE3
0,0,76003,Kareem,Abdul-Jabbar,Milwaukee Bucks,All-Defensive Team,2,1969,NaN,NaN,1610612749,Award,Kia Motors,KIADT,NaN


With that now we can add three more columns 'All_NBA_First_Team', 'All_NBA_Second_Team' and 'All_NBA_Third_Team' that indicates true if the player has been selected in that season in the All NBA First Team, All NBA Second Team or All NBA Third Team.

In [514]:
allNBA = extract_all_nba_teams(awards_not_clean)


allNBA.head(3)

,nbapersonid,season,All_NBA_First_Team,All_NBA_Second_Team,All_NBA_Third_Team
11,76003,1969,False,True,False
12,76003,1970,True,False,False
13,76003,1971,True,False,False


MVP

In [515]:
mvpNBA = extract_mvp_nba(awards_not_clean)

mvpNBA.head(5)

,nbapersonid,season,Most_Valuable_Player
30,76003,1970,True
31,76003,1971,True
32,76003,1973,True
33,76003,1975,True
34,76003,1976,True


DPOY

In [516]:
dpoyNBA = extract_dpoy_nba(awards_not_clean)

dpoyNBA.head(5)

,nbapersonid,season,Defensive_Player_of_The_Year
144,203507,2019,True
762,948,2006,True
852,2199,2011,True
910,76444,1986,True
1392,76631,1984,True


Now we get the all star rosters obtained before and we adapt them to have it in the same format as the other dataframes

In [517]:
all_star_games_rosters2 = transform_all_star_rosters(all_star_games_rosters)


all_star_games_rosters2.head(3)

,nbapersonid,season,all_star_game
0,1628369,2022,True
1,203507,2022,True
2,1628374,2022,True


The same with the df of the all nba players that have been played in the league

In [518]:
nb = nba_players.copy()

nb.rename(columns={'id': 'nbapersonid'}, inplace=True)

nb = nb[['nbapersonid', 'full_name']]

nb.head(2)

,nbapersonid,full_name
0,76001,Alaa Abdelnaby
1,76002,Zaid Abdul-Aziz


Now we perform a full outer join with the data of the players that have been selected in a season in any of the All NBA teams. All NBA First Team, All NBA Second Team or All NBA Third Team. And with the all star games rosters df to have all this information in a single dataframe.

In [519]:
r1 = pd.merge(allNBA, all_star_games_rosters2, on=['nbapersonid', 'season'], how='outer')

r1.head(3)

,nbapersonid,season,All_NBA_First_Team,All_NBA_Second_Team,All_NBA_Third_Team,all_star_game
0,76003,1969,False,True,False,NaN
1,76003,1970,True,False,False,NaN
2,76003,1971,True,False,False,NaN


In [520]:
r2 = pd.merge(r1, dpoyNBA, on=['nbapersonid', 'season'], how='outer')

r2.head(3)

,nbapersonid,season,All_NBA_First_Team,All_NBA_Second_Team,All_NBA_Third_Team,all_star_game,Defensive_Player_of_The_Year
0,76003,1969,False,True,False,NaN,NaN
1,76003,1970,True,False,False,NaN,NaN
2,76003,1971,True,False,False,NaN,NaN


In [521]:
r3 = pd.merge(r2, mvpNBA, on=['nbapersonid', 'season'], how='outer')

r3.head(3)

,nbapersonid,season,All_NBA_First_Team,All_NBA_Second_Team,All_NBA_Third_Team,all_star_game,Defensive_Player_of_The_Year,Most_Valuable_Player
0,76003,1969,False,True,False,NaN,NaN,NaN
1,76003,1970,True,False,False,NaN,NaN,True
2,76003,1971,True,False,False,NaN,NaN,True


After that we convert some columns to binary where 0 stands for False and 1 for True.

In [522]:
r4 = convert_columns_to_binary(r3)

Finally since the oldest season from which we have data for the all star games is the 1983-1984 season and the first Defensive player of the year was Sidney Moncrief in the 1982-1983. We will export only the awards data from 1983-1984 season until now.

In [523]:
awards = r4[r4['season'] >= 1983]

# Final Output

In [524]:
player_stats.to_csv('player_stats_clean.csv', index= False)
player_stats.head(20)

,nbapersonid,player,draftyear,draftpick,season,nbateamid,team,games,games_start,mins,fgm,fga,fgp,fgm3,fga3,fgp3,fgm2,fga2,fgp2,efg,ftm,fta,ftp,off_reb,def_reb,tot_reb,ast,steals,blocks,tov,tot_fouls,points
0,76001,Alaa Abdelnaby,1990,25,1990,1610612757,POR,43,0.00,290.00,55,116,0.47,0.00,0.00,0.00,55.00,116.00,0.47,0.47,25,44,0.57,27.00,62.00,89.00,12,4.00,12.00,22.00,39,135
1,76001,Alaa Abdelnaby,1990,25,1991,1610612757,POR,71,1.00,934.00,178,361,0.49,0.00,0.00,0.00,178.00,361.00,0.49,0.49,76,101,0.75,81.00,179.00,260.00,30,25.00,16.00,66.00,132,432
2,76001,Alaa Abdelnaby,1990,25,1992,1610612749,MIL,12,0.00,159.00,26,56,0.46,0.00,1.00,0.00,26.00,55.00,0.47,0.46,12,16,0.75,12.00,25.00,37.00,10,6.00,4.00,13.00,24,64
3,76001,Alaa Abdelnaby,1990,25,1992,1610612738,BOS,63,52.00,1152.00,219,417,0.53,0.00,0.00,0.00,219.00,417.00,0.53,0.53,76,100,0.76,114.00,186.00,300.00,17,19.00,22.00,84.00,165,514
4,76001,Alaa Abdelnaby,1990,25,1993,1610612738,BOS,13,0.00,159.00,24,55,0.44,0.00,0.00,0.00,24.00,55.00,0.44,0.44,16,25,0.64,12.00,34.00,46.00,3,2.00,3.00,17.00,20,64
5,76001,Alaa Abdelnaby,1990,25,1994,1610612758,SAC,51,0.00,476.00,117,220,0.53,0.00,2.00,0.00,117.00,218.00,0.54,0.53,20,35,0.57,34.00,72.00,106.00,13,15.00,12.00,40.00,102,254
6,76001,Alaa Abdelnaby,1990,25,1994,1610612755,PHL,3,0.00,30.00,1,11,0.09,0.00,0.00,0.00,1.00,11.00,0.09,0.09,0,0,0.00,3.00,5.00,8.00,0,0.00,0.00,5.00,2,2
33,76003,Kareem Abdul-Jabbar,1969,1,1983,1610612747,LAL,80,80.00,2622.00,716,1238,0.58,0.00,1.00,0.00,716.00,1237.00,0.58,0.58,285,394,0.72,169.00,418.00,587.00,211,55.00,143.00,221.00,211,1717
34,76003,Kareem Abdul-Jabbar,1969,1,1984,1610612747,LAL,79,79.00,2630.00,723,1207,0.60,0.00,1.00,0.00,723.00,1206.00,0.60,0.60,289,395,0.73,162.00,460.00,622.00,249,63.00,162.00,197.00,238,1735
35,76003,Kareem Abdul-Jabbar,1969,1,1985,1610612747,LAL,79,79.00,2629.00,755,1338,0.56,0.00,2.00,0.00,755.00,1336.00,0.57,0.56,336,439,0.77,133.00,345.00,478.00,280,67.00,130.00,203.00,248,1846


In [525]:
awards.to_csv('awards_clean.csv', index= False)
awards.head(20)

,nbapersonid,season,All_NBA_First_Team,All_NBA_Second_Team,All_NBA_Third_Team,all_star_game,Defensive_Player_of_The_Year,Most_Valuable_Player
12,76003,1983,1,0,0,True,0,0
13,76003,1984,0,1,0,True,0,0
14,76003,1985,1,0,0,True,0,0
15,200746,2010,0,0,1,False,0,0
16,200746,2013,0,0,1,True,0,0
17,200746,2014,0,1,0,True,0,0
18,200746,2015,0,0,1,True,0,0
19,200746,2017,0,1,0,True,0,0
20,951,2000,0,0,1,True,0,0
21,951,2004,0,1,0,True,0,0


# Data Quality Checking

In [526]:
def check_points_formula(df):
    condition_met = df['points'] == (2 * df['fgm2'] + 3 * df['fgm3'] + df['ftm'])
    return condition_met

In [527]:
def check_fouls_per_game(df):
    condition_met = (df['tot_fouls'] / df['games']) <= 6
    return condition_met

In [528]:
def check_minutes_per_game(df):
    condition_met = (df['mins'] / df['games']) < 48
    return condition_met

In [529]:
def check_games_started(df):
    condition_met = df['games_start'] <= df['games']
    return condition_met

In [530]:
def check_percentage_columns(df):
    percentage_cols = ['fgp', 'fgp3', 'fgp2']
    condition_met = (df[percentage_cols].apply(lambda row: (row >= 0) & (row <= 1), axis=1)).all(axis=1)
    return condition_met

In [531]:
def check_player_conditions(df):
    check_player_stats = df.copy()
    
    check_player_stats['points_condition_met'] = check_points_formula(check_player_stats)
    check_player_stats['fouls_condition_met'] = check_fouls_per_game(check_player_stats)
    check_player_stats['mins_condition_met'] = check_minutes_per_game(check_player_stats)
    check_player_stats['games_condition_met'] = check_games_started(check_player_stats)
    check_player_stats['percentages_condition_met'] = check_percentage_columns(check_player_stats)
    
    points_check = check_player_stats[check_player_stats['points_condition_met'] == False]
    fouls_check = check_player_stats[check_player_stats['fouls_condition_met'] == False]
    mins_check = check_player_stats[check_player_stats['mins_condition_met'] == False]
    games_check = check_player_stats[check_player_stats['games_condition_met'] == False]
    percentages_check = check_player_stats[check_player_stats['percentages_condition_met'] == False]
    
    if points_check.empty and fouls_check.empty and mins_check.empty and games_check.empty and percentages_check.empty:
        print("All checks passed successfully.")
    else:
        print("Some checks failed.")

In [532]:
check_player_conditions(player_stats)

All checks passed successfully.
